In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
import math
from IPython.display import FileLink

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    #for filename in filenames: print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input
/kaggle/input/hf-round2-train
/kaggle/input/hf-round2-train/hf_round2_train
/kaggle/input/hf-round2-train/hf_round2_train/hf_round2_train


In [2]:
# train
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231754/round2/hf_round2_train.zip

# label
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231754/round2/hf_round2_train.txt

# arrythmia
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231754/round2/hf_round2_arrythmia.txt

wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)
--2019-12-18 14:18:31--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231754/round2/hf_round2_train.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748182618 (714M) [application/zip]
Saving to: ‘hf_round2_train.zip’

hf_round2_train.zip 100%[===================>] 713.52M  1.41MB/s    in 9m 22s  

2019-12-18 14:27:54 (1.27 MB/s) - ‘hf_round2_train.zip’ saved [748182618/748182618]

wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)
--2019-12-18 14:27:55--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/231754/round2/hf_round2_train.txt
Resolving tianchi-competition.o

In [3]:
#check diseases
disease_dict = {}
with open('/kaggle/input/hf-round2-train/hf_round2_arrythmia.txt','r') as dict_f:
    for count, line in enumerate(dict_f):
        disease_dict[line.replace('\n','')] = count
disease_dict

{'QRS低电压': 0,
 '电轴右偏': 1,
 '起搏心律': 2,
 'T波改变': 3,
 '电轴左偏': 4,
 '心房颤动': 5,
 '非特异性ST段异常': 6,
 '下壁异常Q波': 7,
 '前间壁R波递增不良': 8,
 'ST段改变': 9,
 '一度房室传导阻滞': 10,
 '左束支传导阻滞': 11,
 '右束支传导阻滞': 12,
 '完全性左束支传导阻滞': 13,
 '左前分支传导阻滞': 14,
 '右心房扩大': 15,
 '短PR间期': 16,
 '左心室高电压': 17,
 '窦性心动过缓': 18,
 '早期复极化': 19,
 '窦性心律': 20,
 '融合波': 21,
 'ST-T改变': 22,
 '非特异性ST段与T波异常': 23,
 '快心室率': 24,
 '非特异性T波异常': 25,
 '室性早搏': 26,
 '房性早搏': 27,
 '窦性心律不齐': 28,
 '完全性右束支传导阻滞': 29,
 '窦性心动过速': 30,
 '不完全性右束支传导阻滞': 31,
 '顺钟向转位': 32,
 '逆钟向转位': 33}

In [4]:
#load labels

labels = []
with open('/kaggle/input/hf-round2-train/hf_round2_train.txt','r') as f:
    for count, line in tqdm(enumerate(f)):
        datas = line.split()
        diseases = []
        exist_age = 0
        exist_sex = 0
        if datas[1].isdigit():  exist_age = 1
        if datas[1] == 'MALE' or datas[1] == 'FEMALE' : exist_sex = 1
        if len(datas) > 2 and (datas[2] == 'MALE' or datas[2] == 'FEMALE') : exist_sex = 1
        for disease in datas[1+exist_age+exist_sex:]:
            diseases.append(disease_dict[disease])
        labels.append(diseases)

#transfer labels to one-hot
hot_labels = []
for label in labels:
    hot = [0]*len(disease_dict)
    for disease in label:
        hot[disease] = 1
    hot_labels.append(hot)
hot_labels = torch.FloatTensor(hot_labels)
hot_labels.shape

num_dict = {}
for dis in labels:
    for di in dis:
        num_dict[di] = num_dict.get(di,0) + 1
#dict(sorted(num_dict.items()))
num_dict

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


{20: 9501,
 3: 3479,
 30: 4895,
 22: 299,
 18: 5264,
 27: 314,
 28: 901,
 1: 1124,
 26: 543,
 29: 418,
 12: 551,
 10: 142,
 17: 414,
 4: 1124,
 32: 35,
 16: 23,
 9: 286,
 5: 120,
 24: 29,
 7: 52,
 8: 16,
 31: 126,
 25: 34,
 19: 22,
 15: 32,
 33: 60,
 13: 25,
 11: 25,
 6: 64,
 14: 35,
 2: 16,
 23: 16,
 21: 7,
 0: 3}

In [5]:
train_data = []
for dirname, dirs, filenames in os.walk('/kaggle/input/hf-round2-train/hf_round2_train'):
    #filenames = sorted(filenames)
    filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    for file_count, filename in tqdm(enumerate(filenames)):
        address = os.path.join(dirname, filename)
        #if file_count == 1000: break
        with open(address,'r') as f:
            individual = []
            for line, v in enumerate(f):
                if line == 0: continue
                v = v.split()
                #print(v)
                individual.append([float(value) for value in v])
                del v
            individual = torch.FloatTensor(individual).transpose(0,1)
            #train_data[file_count] = individual
            train_data.append(individual)
            del individual
            f.close()
        #break
train_data = torch.stack(train_data)
torch.save(train_data, 'train_data.tensor')
torch.save(hot_labels, 'hot_labels.tensor')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """
